In [1]:

import sys
import os
import json
import math
import random
import requests
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional, Callable
import warnings
warnings.filterwarnings('ignore')

print("🛠️ Welcome to Agent Tools & Actions!")
print("=" * 60)

# Set up our workspace
current_dir = os.getcwd()
if 'notebooks' in current_dir:
    project_dir = os.path.dirname(current_dir)
else:
    project_dir = current_dir

print(f"🔧 Environment Setup Complete!")
print(f"📍 Project directory: {project_dir}")
print(f"📅 Session started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print(f"\n🎯 Today's Learning Journey:")
print(f"   🔨 1. Understand What Tools Are")
print(f"   🛠️ 2. Build Individual Tools") 
print(f"   🤖 3. Create Tool-Using Agents")
print(f"   ⚡ 4. Test Real-World Actions")

print(f"\n🚀 Let's give our agents some superpowers!")

🛠️ Welcome to Agent Tools & Actions!
🔧 Environment Setup Complete!
📍 Project directory: c:\Users\Sarthak\OneDrive\Documents\Desktop\Python projects\genai-workshop
📅 Session started: 2025-07-21 11:00:25

🎯 Today's Learning Journey:
   🔨 1. Understand What Tools Are
   🛠️ 2. Build Individual Tools
   🤖 3. Create Tool-Using Agents
   ⚡ 4. Test Real-World Actions

🚀 Let's give our agents some superpowers!


In [4]:
import math
import re

def calculator_tool(expression: str) -> dict:
    """
    A calculator tool that agents can use to solve math problems.
    
    Args:
        expression: Math expression as string (e.g., "2 + 2", "sqrt(16)", "2^8")
        
    Returns:
        Dictionary with result and metadata
    """
    print(f"🧮 CALCULATOR TOOL: Processing '{expression}'")
    
    try:
        # Clean and prepare the expression
        clean_expr = expression.strip()
        
        # Handle percentage calculations first
        if '%' in clean_expr:
            # Handle patterns like "15% of 200", "15% * 200", or "20% of 150"
            import re
            # More flexible regex pattern
            percent_match = re.search(r'(\d+(?:\.\d+)?)\s*%\s*(?:of|×|\*|x)\s*(\d+(?:\.\d+)?)', clean_expr, re.IGNORECASE)
            if percent_match:
                percent = float(percent_match.group(1))
                number = float(percent_match.group(2))
                result = (percent / 100) * number
                return {
                    'tool': 'calculator',
                    'input': expression,
                    'result': result,
                    'formatted_result': f"{result:,.2f}",
                    'success': True,
                    'error': None
                }
            else:
                # If no "of" pattern, maybe it's just "15%" which is invalid alone
                return {
                    'tool': 'calculator',
                    'input': expression,
                    'result': None,
                    'formatted_result': None,
                    'success': False,
                    'error': "Percentage calculation needs format like '15% of 200'"
                }
        
        # Replace ^ with ** for Python power operator
        clean_expr = clean_expr.replace('^', '**')
        
        # Add support for common math functions
        clean_expr = clean_expr.replace('sqrt', 'math.sqrt')
        clean_expr = clean_expr.replace('sin', 'math.sin')
        clean_expr = clean_expr.replace('cos', 'math.cos')
        clean_expr = clean_expr.replace('log', 'math.log')
        
        # Safe evaluation using restricted globals
        allowed_names = {
            "__builtins__": {},
            "math": math,
            "abs": abs,
            "round": round,
            "min": min,
            "max": max,
            "sum": sum
        }
        
        # Calculate the result safely
        result = eval(clean_expr, allowed_names, {})
        
        return {
            'tool': 'calculator',
            'input': expression,
            'result': result,
            'formatted_result': f"{result:,.2f}" if isinstance(result, float) else str(result),
            'success': True,
            'error': None
        }
        
    except Exception as e:
        return {
            'tool': 'calculator',
            'input': expression,
            'result': None,
            'formatted_result': None,
            'success': False,
            'error': f"Calculation error: {str(e)}"
        }

# 🧪 Test the calculator tool
print("🧪 TESTING CALCULATOR TOOL")
print("=" * 40)

test_calculations = [
    "2 + 2",
    "15 * 9", 
    "100 / 3",
    "2^8",
    "sqrt(144)",
    "1% of 200"  # This should work now!
]

for calc in test_calculations:
    print(f"\n📥 INPUT: {calc}")
    result = calculator_tool(calc)
    
    if result['success']:
        print(f"✅ RESULT: {result['formatted_result']}")
    else:
        print(f"❌ ERROR: {result['error']}")

print(f"\n💡 The calculator tool can handle basic math and returns structured results!")
print(f"🛡️ It gracefully handles errors without crashing!")

🧪 TESTING CALCULATOR TOOL

📥 INPUT: 2 + 2
🧮 CALCULATOR TOOL: Processing '2 + 2'
✅ RESULT: 4

📥 INPUT: 15 * 9
🧮 CALCULATOR TOOL: Processing '15 * 9'
✅ RESULT: 135

📥 INPUT: 100 / 3
🧮 CALCULATOR TOOL: Processing '100 / 3'
✅ RESULT: 33.33

📥 INPUT: 2^8
🧮 CALCULATOR TOOL: Processing '2^8'
✅ RESULT: 256

📥 INPUT: sqrt(144)
🧮 CALCULATOR TOOL: Processing 'sqrt(144)'
✅ RESULT: 12.00

📥 INPUT: 1% of 200
🧮 CALCULATOR TOOL: Processing '1% of 200'
✅ RESULT: 2.00

💡 The calculator tool can handle basic math and returns structured results!
🛡️ It gracefully handles errors without crashing!


In [5]:
import os
from datetime import datetime

def file_manager_tool(action: str, filepath: str = None, content: str = None) -> dict:
    """
    A file management tool that agents can use to work with files.
    
    Args:
        action: 'read', 'write', 'list', 'exists', 'delete'
        filepath: Path to the file (required for most actions)
        content: Content to write (required for 'write' action)
        
    Returns:
        Dictionary with result and metadata
    """
    print(f"📂 FILE MANAGER: {action.upper()} operation")
    
    try:
        if action == 'read':
            if not filepath:
                raise ValueError("Filepath required for read operation")
            
            with open(filepath, 'r') as f:
                file_content = f.read()
            
            return {
                'tool': 'file_manager',
                'action': action,
                'filepath': filepath,
                'content': file_content,
                'success': True,
                'message': f"Successfully read {len(file_content)} characters",
                'error': None
            }
            
        elif action == 'write':
            if not filepath or content is None:
                raise ValueError("Filepath and content required for write operation")
            
            # Create directory if it doesn't exist (only if there's a directory path)
            directory = os.path.dirname(filepath)
            if directory:  # Only create directory if path contains a directory
                os.makedirs(directory, exist_ok=True)
            
            with open(filepath, 'w') as f:
                  f.write(content)
            
            return {
                'tool': 'file_manager',
                'action': action,
                'filepath': filepath,
                'content': content,
                'success': True,
                'message': f"Successfully wrote {len(content)} characters to {filepath}",
                'error': None
            }
            
        elif action == 'exists':
            if not filepath:
                raise ValueError("Filepath required for exists operation")
            
            exists = os.path.exists(filepath)
            
            return {
                'tool': 'file_manager', 
                'action': action,
                'filepath': filepath,
                'exists': exists,
                'success': True,
                'message': f"File {'exists' if exists else 'does not exist'}",
                'error': None
            }
            
        elif action == 'list':
            directory = filepath or '.'
            files = os.listdir(directory)
            
            return {
                'tool': 'file_manager',
                'action': action,
                'directory': directory,
                'files': files,
                'count': len(files),
                'success': True,
                'message': f"Found {len(files)} items in {directory}",
                'error': None
            }
            
        else:
            raise ValueError(f"Unknown action: {action}")
            
    except Exception as e:
        return {
            'tool': 'file_manager',
            'action': action,
            'filepath': filepath,
            'success': False,
            'message': None,
            'error': f"File operation error: {str(e)}"
        }

# 🧪 Test the file manager tool
print("🧪 TESTING FILE MANAGER TOOL")
print("=" * 40)

# Test creating a file
test_content = f"Hello from AI Agent!\nCreated at: {datetime.now()}\nThis file was created by an agent tool!"

print(f"\n📝 Creating test file...")
write_result = file_manager_tool('write', 'test_agent_file.txt', test_content)
if write_result['success']:
    print(f"✅ {write_result['message']}")
else:
    print(f"❌ {write_result['error']}")

# Test reading the file back
print(f"\n📖 Reading test file...")
read_result = file_manager_tool('read', 'test_agent_file.txt')
if read_result['success']:
    print(f"✅ {read_result['message']}")
    print(f"📄 Content preview: {read_result['content'][:50]}...")
else:
    print(f"❌ {read_result['error']}")

# Test checking if file exists
print(f"\n🔍 Checking if file exists...")
exists_result = file_manager_tool('exists', 'test_agent_file.txt')
print(f"✅ {exists_result['message']}")

print(f"\n💡 The file manager tool gives agents the ability to persist information!")
print(f"🛡️ It handles errors and provides clear feedback about operations!")

🧪 TESTING FILE MANAGER TOOL

📝 Creating test file...
📂 FILE MANAGER: WRITE operation
✅ Successfully wrote 99 characters to test_agent_file.txt

📖 Reading test file...
📂 FILE MANAGER: READ operation
✅ Successfully read 99 characters
📄 Content preview: Hello from AI Agent!
Created at: 2025-07-21 11:10:...

🔍 Checking if file exists...
📂 FILE MANAGER: EXISTS operation
✅ File exists

💡 The file manager tool gives agents the ability to persist information!
🛡️ It handles errors and provides clear feedback about operations!


In [6]:
import time
from datetime import datetime, timedelta

# Global timer state (in a real system, this would be more sophisticated)
timer_state = {
    'active_timers': {},
    'completed_timers': []
}

def timer_tool(action: str, duration_minutes: int = None, timer_name: str = "default") -> dict:
    """
    A timer tool that agents can use for time management.
    
    Args:
        action: 'start', 'check', 'stop', 'list'
        duration_minutes: How long to run the timer (for start action)
        timer_name: Name to identify this timer
        
    Returns:
        Dictionary with result and metadata
    """
    print(f"⏰ TIMER TOOL: {action.upper()} operation")
    
    try:
        if action == 'start':
            if duration_minutes is None:
                raise ValueError("Duration required for start action")
            
            start_time = datetime.now()
            end_time = start_time + timedelta(minutes=duration_minutes)
            
            timer_state['active_timers'][timer_name] = {
                'start_time': start_time,
                'end_time': end_time,
                'duration_minutes': duration_minutes,
                'status': 'running'
            }
            
            return {
                'tool': 'timer',
                'action': action,
                'timer_name': timer_name,
                'duration_minutes': duration_minutes,
                'start_time': start_time.strftime('%H:%M:%S'),
                'end_time': end_time.strftime('%H:%M:%S'),
                'success': True,
                'message': f"Timer '{timer_name}' started for {duration_minutes} minutes",
                'error': None
            }
            
        elif action == 'check':
            if timer_name not in timer_state['active_timers']:
                return {
                    'tool': 'timer',
                    'action': action,
                    'timer_name': timer_name,
                    'success': False,
                    'message': None,
                    'error': f"No active timer named '{timer_name}'"
                }
            
            timer = timer_state['active_timers'][timer_name]
            now = datetime.now()
            elapsed = now - timer['start_time']
            remaining = timer['end_time'] - now
            
            is_finished = now >= timer['end_time']
            
            if is_finished and timer['status'] == 'running':
                # Move to completed timers
                timer['status'] = 'completed'
                timer_state['completed_timers'].append(timer)
                del timer_state['active_timers'][timer_name]
                
                return {
                    'tool': 'timer',
                    'action': action,
                    'timer_name': timer_name,
                    'finished': True,
                    'elapsed_minutes': timer['duration_minutes'],
                    'success': True,
                    'message': f"🎉 Timer '{timer_name}' completed!",
                    'error': None
                }
            else:
                return {
                    'tool': 'timer', 
                    'action': action,
                    'timer_name': timer_name,
                    'finished': False,
                    'elapsed_minutes': int(elapsed.total_seconds() / 60),
                    'remaining_minutes': int(remaining.total_seconds() / 60),
                    'success': True,
                    'message': f"Timer '{timer_name}' running: {int(remaining.total_seconds() / 60)} minutes left",
                    'error': None
                }
                
        elif action == 'list':
            active_count = len(timer_state['active_timers'])
            completed_count = len(timer_state['completed_timers'])
            
            return {
                'tool': 'timer',
                'action': action,
                'active_timers': list(timer_state['active_timers'].keys()),
                'active_count': active_count,
                'completed_count': completed_count,
                'success': True,
                'message': f"Found {active_count} active and {completed_count} completed timers",
                'error': None
            }
            
        else:
            raise ValueError(f"Unknown action: {action}")
            
    except Exception as e:
        return {
            'tool': 'timer',
            'action': action,
            'timer_name': timer_name,
            'success': False,
            'message': None,
            'error': f"Timer error: {str(e)}"
        }

# 🧪 Test the timer tool
print("🧪 TESTING TIMER TOOL")
print("=" * 40)

# Start a very short timer for testing (10 seconds = 0.17 minutes)
print(f"\n⏰ Starting a 10-second test timer...")
start_result = timer_tool('start', duration_minutes=0.17, timer_name='test_timer')
if start_result['success']:
    print(f"✅ {start_result['message']}")
    print(f"🕐 Started at: {start_result['start_time']}")
    print(f"🕐 Will end at: {start_result['end_time']}")
else:
    print(f"❌ {start_result['error']}")

# Check timer status immediately
print(f"\n🔍 Checking timer status...")
check_result = timer_tool('check', timer_name='test_timer')
if check_result['success']:
    print(f"✅ {check_result['message']}")
    if not check_result['finished']:
        print(f"⏳ Time remaining: {check_result['remaining_minutes']} minutes")

# List all timers
print(f"\n📋 Listing all timers...")
list_result = timer_tool('list')
print(f"✅ {list_result['message']}")
print(f"📊 Active timers: {list_result['active_timers']}")

print(f"\n💡 The timer tool enables agents to manage time and schedules!")
print(f"⚡ Perfect for implementing Pomodoro sessions, reminders, and deadlines!")

🧪 TESTING TIMER TOOL

⏰ Starting a 10-second test timer...
⏰ TIMER TOOL: START operation
✅ Timer 'test_timer' started for 0.17 minutes
🕐 Started at: 11:15:34
🕐 Will end at: 11:15:44

🔍 Checking timer status...
⏰ TIMER TOOL: CHECK operation
✅ Timer 'test_timer' running: 0 minutes left
⏳ Time remaining: 0 minutes

📋 Listing all timers...
⏰ TIMER TOOL: LIST operation
✅ Found 1 active and 0 completed timers
📊 Active timers: ['test_timer']

💡 The timer tool enables agents to manage time and schedules!
⚡ Perfect for implementing Pomodoro sessions, reminders, and deadlines!


In [7]:

def tool_using_agent(user_request: str) -> str:
    """
    An agent that can analyze requests and use appropriate tools.
    
    This demonstrates the core pattern of tool-using agents:
    1. Understand the request
    2. Choose the right tool  
    3. Use the tool
    4. Return results in natural language
    """
    print(f"\n🤖 AGENT PROCESSING: '{user_request}'")
    print("=" * 50)
    
    # STEP 1: Analyze the request to understand intent
    request_lower = user_request.lower()
    
    # STEP 2: Choose the appropriate tool based on keywords and patterns
    if any(word in request_lower for word in ['calculate', 'math', 'plus', 'minus', '+', '-', '*', '/', '^', 'sqrt', '%']):
        print("🔍 INTENT DETECTED: Mathematical calculation needed")
        print("🛠️ TOOL SELECTED: Calculator")
        
        # Extract the math expression (improved approach)
        import re
        
        # Try different patterns to extract math expressions
        if 'calculate' in request_lower:
            # "Calculate 25 * 4" -> "25 * 4"
            # Use case-insensitive split by using request_lower
            math_part = user_request.lower().split('calculate')[-1].strip()
            # But preserve original case for numbers/operators
            # Extract just the math part with regex to maintain proper formatting
            math_match = re.search(r'calculate\s+(.*)', user_request, re.IGNORECASE)
            if math_match:
                math_part = math_match.group(1).strip()
            else:
                math_part = user_request.lower().split('calculate')[-1].strip()
        elif 'what' in request_lower and ('of' in request_lower or '*' in user_request):
            # "What's 20% of 150?" -> "20% of 150"
            # Extract percentage calculation
            percent_match = re.search(r'(\d+(?:\.\d+)?)\s*%\s*(?:of|×|\*)\s*(\d+(?:\.\d+)?)', user_request)
            if percent_match:
                math_part = f"{percent_match.group(1)}% of {percent_match.group(2)}"
            else:
                # Try to find numbers and operators
                numbers_and_ops = re.findall(r'[\d.]+|[+\-*/^%()]|sqrt|of', user_request)
                math_part = ' '.join(numbers_and_ops)
        elif any(op in user_request for op in ['+', '-', '*', '/', '^']):
            # Direct math expression like "25 * 4"
            math_part = user_request.strip()
        else:
            # Try to extract numbers and math terms
            numbers_and_ops = re.findall(r'[\d.]+|[+\-*/^%()]|sqrt|of', user_request)
            if len(numbers_and_ops) >= 3:  # At least number operator number
                math_part = ' '.join(numbers_and_ops)
            else:
                math_part = user_request.strip()
            
        result = calculator_tool(math_part)
        
        if result['success']:
            return f"🧮 I calculated that for you: {math_part} = {result['formatted_result']}"
        else:
            return f"❌ I had trouble with that calculation: {result['error']}"
    
    elif any(word in request_lower for word in ['save', 'write', 'store', 'remember', 'file']):
        print("🔍 INTENT DETECTED: File operation needed")
        print("🛠️ TOOL SELECTED: File Manager")
        
        # Simple content extraction (in real agent, this would be more sophisticated)
        if 'save' in request_lower or 'remember' in request_lower:
            content = user_request
            filename = f"agent_note_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
            
            result = file_manager_tool('write', filename, content)
            
            if result['success']:
                return f"📁 I saved that information to {filename} for you!"
            else:
                return f"❌ I couldn't save the file: {result['error']}"
        else:
            return f"📂 I can help with file operations! Try asking me to save something."
    
    elif any(word in request_lower for word in ['timer', 'time', 'minutes', 'session', 'focus']):
        print("🔍 INTENT DETECTED: Timer operation needed")
        print("🛠️ TOOL SELECTED: Timer")
        
        # Extract duration (simplified approach)
        import re
        duration_match = re.search(r'(\d+)\s*min', request_lower)
        
        if duration_match:
            duration = int(duration_match.group(1))
            session_name = "focus_session"
            
            result = timer_tool('start', duration, session_name)
            
            if result['success']:
                return f"⏰ I started a {duration}-minute timer for you! It will end at {result['end_time']}."
            else:
                return f"❌ I couldn't start the timer: {result['error']}"
        else:
            return f"⏰ I can help with timers! Try asking for a specific duration like '25 minutes'."
    
    else:
        print("🔍 INTENT DETECTED: General conversation")
        print("🛠️ TOOL SELECTED: None (direct response)")
        
        return f"🤖 I can help you with calculations, file management, and timers! What would you like me to do?"

# 🧪 Test the tool-using agent
print("🧪 TESTING TOOL-USING AGENT")
print("=" * 50)

test_requests = [
    "Calculate 25 * 4",
    "Save this note: Remember to study Python functions tomorrow", 
    "Start a 25 minute focus session",
    "Hello, what can you do?"
]

for request in test_requests:
    print(f"\n👤 USER: {request}")
    response = tool_using_agent(request)
    print(f"🤖 AGENT: {response}")
    print()

print("🎉 SUCCESS! Our agent can now:")
print("✅ Understand different types of requests")
print("✅ Choose the appropriate tool")  
print("✅ Use tools to take real actions")
print("✅ Respond in natural language")
print("\n💡 This is the foundation of all intelligent agent systems!")

🧪 TESTING TOOL-USING AGENT

👤 USER: Calculate 25 * 4

🤖 AGENT PROCESSING: 'Calculate 25 * 4'
🔍 INTENT DETECTED: Mathematical calculation needed
🛠️ TOOL SELECTED: Calculator
🧮 CALCULATOR TOOL: Processing '25 * 4'
🤖 AGENT: 🧮 I calculated that for you: 25 * 4 = 100


👤 USER: Save this note: Remember to study Python functions tomorrow

🤖 AGENT PROCESSING: 'Save this note: Remember to study Python functions tomorrow'
🔍 INTENT DETECTED: File operation needed
🛠️ TOOL SELECTED: File Manager
📂 FILE MANAGER: WRITE operation
🤖 AGENT: 📁 I saved that information to agent_note_20250721_120229.txt for you!


👤 USER: Start a 25 minute focus session

🤖 AGENT PROCESSING: 'Start a 25 minute focus session'
🔍 INTENT DETECTED: Timer operation needed
🛠️ TOOL SELECTED: Timer
⏰ TIMER TOOL: START operation
🤖 AGENT: ⏰ I started a 25-minute timer for you! It will end at 12:27:29.


👤 USER: Hello, what can you do?

🤖 AGENT PROCESSING: 'Hello, what can you do?'
🔍 INTENT DETECTED: General conversation
🛠️ TOOL SELECT

In [8]:
print("🧪 COMPREHENSIVE TOOL VALIDATION")
print("=" * 60)

# Test 1: Calculator Tool - Various expressions
print("\n📊 TEST 1: Calculator Tool")
print("-" * 30)

calc_tests = [
    "2 + 2",           # Simple addition
    "25 * 4",          # Multiplication  
    "100 / 3",         # Division
    "2^8",             # Power
    "sqrt(144)",       # Square root
    "15% of 200",      # Percentage
    "20% of 150"       # Another percentage
]

for expr in calc_tests:
    result = calculator_tool(expr)
    status = "✅" if result['success'] else "❌"
    output = result['formatted_result'] if result['success'] else result['error']
    print(f"{status} {expr} = {output}")

# Test 2: File Manager Tool  
print("\n📁 TEST 2: File Manager Tool")
print("-" * 30)

test_file = "validation_test.txt"
test_content = "This is a test file created during validation."

# Write test
write_result = file_manager_tool('write', test_file, test_content)
print(f"✅ Write: {write_result['message']}" if write_result['success'] else f"❌ Write failed: {write_result['error']}")

# Read test
read_result = file_manager_tool('read', test_file)
print(f"✅ Read: {read_result['message']}" if read_result['success'] else f"❌ Read failed: {read_result['error']}")

# Exists test
exists_result = file_manager_tool('exists', test_file)
print(f"✅ Exists: {exists_result['message']}")

# Test 3: Timer Tool
print("\n⏰ TEST 3: Timer Tool")
print("-" * 30)

# Start a very short timer
timer_result = timer_tool('start', 0.1, 'validation_timer')  # 6 seconds
print(f"✅ Timer started: {timer_result['message']}" if timer_result['success'] else f"❌ Timer failed: {timer_result['error']}")

# Check timer
check_result = timer_tool('check', timer_name='validation_timer')
print(f"✅ Timer check: {check_result['message']}" if check_result['success'] else f"❌ Timer check failed: {check_result['error']}")

# List timers
list_result = timer_tool('list')
print(f"✅ Timer list: {list_result['message']}")

print(f"\n🎉 ALL TOOLS VALIDATED SUCCESSFULLY!")
print(f"✅ Calculator: Handles math expressions and percentages")
print(f"✅ File Manager: Creates, reads, and manages files")  
print(f"✅ Timer: Manages time-based activities")
print(f"\n🚀 Ready for agent integration!")

🧪 COMPREHENSIVE TOOL VALIDATION

📊 TEST 1: Calculator Tool
------------------------------
🧮 CALCULATOR TOOL: Processing '2 + 2'
✅ 2 + 2 = 4
🧮 CALCULATOR TOOL: Processing '25 * 4'
✅ 25 * 4 = 100
🧮 CALCULATOR TOOL: Processing '100 / 3'
✅ 100 / 3 = 33.33
🧮 CALCULATOR TOOL: Processing '2^8'
✅ 2^8 = 256
🧮 CALCULATOR TOOL: Processing 'sqrt(144)'
✅ sqrt(144) = 12.00
🧮 CALCULATOR TOOL: Processing '15% of 200'
✅ 15% of 200 = 30.00
🧮 CALCULATOR TOOL: Processing '20% of 150'
✅ 20% of 150 = 30.00

📁 TEST 2: File Manager Tool
------------------------------
📂 FILE MANAGER: WRITE operation
✅ Write: Successfully wrote 46 characters to validation_test.txt
📂 FILE MANAGER: READ operation
✅ Read: Successfully read 46 characters
📂 FILE MANAGER: EXISTS operation
✅ Exists: File exists

⏰ TEST 3: Timer Tool
------------------------------
⏰ TIMER TOOL: START operation
✅ Timer started: Timer 'validation_timer' started for 0.1 minutes
⏰ TIMER TOOL: CHECK operation
✅ Timer check: Timer 'validation_timer' running: